In [ ]:
import sqlite3

In [1]:
conn = sqlite3.connect("creditcard.db")

cursor = conn.cursor()
cursor.execute(
    """
    CREATE TABLE  IF NOT EXISTS education(
            "id"	INTEGER,
            "年"	TEXT NOT NULL,
            "月"	TEXT NOT NULL,
            "地區"	TEXT NOT NULL,
            "產業別"	TEXT NOT NULL,
            "教育程度類別"	TEXT NOT NULL,
            "信用卡交易筆數"	INTEGER NOT NULL,
            "信用卡交易金額[新台幣]"	INTEGER NOT NULL,            
            PRIMARY KEY("id" AUTOINCREMENT)
    );
    """
)

cursor.execute(
    """
    CREATE TABLE  IF NOT EXISTS incom(
        "id"	INTEGER,
        "年"	TEXT NOT NULL,
        "月"	TEXT NOT NULL,
        "地區"	TEXT NOT NULL,
        "產業別"	TEXT NOT NULL,
        "年收入"	TEXT NOT NULL,
        "信用卡交易筆數"	INTEGER NOT NULL,
        "信用卡交易金額[新台幣]"	INTEGER NOT NULL,            
        PRIMARY KEY("id" AUTOINCREMENT)
    );
    """
)

cursor.execute(
    """
    CREATE TABLE  IF NOT EXISTS sex(
        "id"	INTEGER,
        "年"	TEXT NOT NULL,
        "月"	TEXT NOT NULL,
        "地區"	TEXT NOT NULL,
        "產業別"	TEXT NOT NULL,
        "性別"	INTEGER NOT NULL,
        "信用卡交易筆數"	INTEGER NOT NULL,
        "信用卡交易金額[新台幣]"	INTEGER NOT NULL,            
        PRIMARY KEY("id" AUTOINCREMENT)
    );
    """
)

cursor.execute(
    """
        CREATE TABLE  IF NOT EXISTS incom(
            "id"	INTEGER,
            "年"	TEXT NOT NULL,
            "月"	TEXT NOT NULL,
            "地區"	TEXT NOT NULL,
            "產業別"	TEXT NOT NULL,
            "職業類別"	TEXT NOT NULL,
            "信用卡交易筆數"	INTEGER NOT NULL,
            "信用卡交易金額[新台幣]"	INTEGER NOT NULL,            
            PRIMARY KEY("id" AUTOINCREMENT)
        );
    """
)

cursor.execute(
    '''
        CREATE TABLE IF NOT EXISTS age(
            "id"    INTEGER,
            "年"    TEXT NOT NULL,
            "月"    TEXT NOT NULL,
            "地區"  TEXT NOT NULL,
            "產業別"    TEXT NOT NULL,
            "性別"  TEXT NOT NULL,
            "年齡層" TEXT NOT NULL,
            "信用卡交易筆數" INTEGER NOT NULL,
            "信用卡交易金額" INTEGER NOT NULL,
            PRIMARY KEY("id" AUTOINCREMENT)
        );
    '''
)



conn.commit()

NameError: name 'sqlite3' is not defined

In [ ]:
import csv
import sqlite3

db_connection = sqlite3.connect("creditcard.db")
cursor = db_connection.cursor()

area = {
    "63000000": "臺北市",
    "64000000": "高雄市",
    "65000000": "新北市",
    "66000000": "臺中市",
    "67000000": "臺南市",
    "68000000": "桃園市",
    "10002000": "宜蘭縣",
    "10004000": "新竹縣",
    "10005000": "苗栗縣",
    "10007000": "彰化縣",
    "10008000": "南投縣",
    "10009000": "雲林縣",
    "10010000": "嘉義縣",
    "10020000": "嘉義市",
    "10013000": "屏東縣",
    "10014000": "臺東縣",
    "10015000": "花蓮縣",
    "10016000": "澎湖縣",
    "10017000": "基隆市",
    "10018000": "新竹市",
    "09020000": "金門縣",
    "09007000": "連江縣",
}

with open("job.csv", "r", encoding="UTF-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        # row['地區'] = row['地區'].replace(row['地區'],area[row['地區']])
        # 遇到非代號會有問題，其餘有對應代號轉換成功
        # 使用get方法，如果找不到對應的鍵，就保持原來的值
        row["地區"] = area.get(row["地區"], row["地區"])

        # print(row)

        sql = """
            INSERT INTO job (年,月,地區,產業別,職業類別,信用卡交易筆數,'信用卡交易金額[新台幣]')VALUES (?,?,?,?,?,?,?)
            """
        values = (
            row["年"],
            row["月"],
            row["地區"],
            row["產業別"],
            row["職業類別"],
            row["信用卡交易筆數"],
            row["信用卡交易金額[新台幣]"],
        )

        cursor.execute(sql, values)


db_connection.commit()
db_connection.close()